## Step 1 — Load Configuration & Base Dataset

In this step, we load the main configuration file (`config.yaml`) to access all project paths, and then import the base dataset `books_clustered_final.csv` from the `data/clean` directory.

This dataset contains the books used for clustering in the previous step. It will serve as the foundation for enriching missing information such as ratings and genres using external data sources.


In [ ]:
# ============================================================
# Step 1 — Load Configuration & Base Dataset
# ============================================================
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
import pandas as pd
from pathlib import Path
from functions import load_config, ensure_directories

# --- Load configuration from project root ---
config_path = Path("..") / "config.yaml"
config = load_config(config_path)

# --- Ensure all folders exist ---
ensure_directories(config["paths"])

# --- Load base dataset ---
data_clean_path = Path("..") / config["paths"]["data_clean"]
input_file = data_clean_path / "books_clustered_final.csv"

df_main = pd.read_csv(input_file)

print(f"Dataset loaded successfully: {df_main.shape}")
df_main.head(3)


## Step 2 — Load Goodreads Dataset (Kaggle goodbooks-10k)

In this step, we load the `books.csv` file from the Kaggle dataset “goodbooks-10k”.  
This dataset contains around 10 000 books with standardized metadata such as title, author, average rating, number of ratings, and publication year.  
It is a lighter and cleaner dataset than the previous BrightData version and aligns well with our book titles.



In [ ]:
# ============================================================
# Step 2 — Load Goodreads Dataset (Kaggle goodbooks-10k)
# ============================================================

import pandas as pd
from pathlib import Path

# Define path (using config paths)
data_raw_path = Path("..") / "data" / "raw"
goodreads_file = data_raw_path / "books.csv"  # rename your downloaded books.csv to this

# Load dataset
df_goodreads = pd.read_csv(goodreads_file)
print(f"Kaggle Goodreads dataset loaded: {df_goodreads.shape}")

# Display available columns
print("Columns:", df_goodreads.columns.tolist())

# Preview
df_goodreads.head(3)


In [ ]:
print(df_goodreads.columns.tolist())


## Step 3 — Preprocess Titles & Authors for Merging

Before merging both datasets, we standardize and align the column names used as matching keys.

In the Kaggle dataset, the relevant columns are:
- `title` → book title  
- `authors` → author name(s)  
- `average_rating` → Goodreads average rating  
- `ratings_count` → total number of ratings  
- `original_publication_year` → publication year  
- `image_url` → cover image

We will:
1. Keep only these relevant columns.  
2. Rename them for consistency.  
3. Normalize `title` and `author` text to lowercase for reliable matching.


In [ ]:
# ============================================================
# Step 3 — Preprocess Titles & Authors for Merging
# ============================================================

# Select and rename relevant columns
cols_to_keep = [
    "title",
    "authors",
    "average_rating",
    "ratings_count",
    "original_publication_year",
    "image_url"
]

df_goodreads = df_goodreads[cols_to_keep].rename(columns={
    "authors": "author",
    "average_rating": "avg_rating_goodreads",
    "ratings_count": "ratings_count_goodreads",
    "original_publication_year": "published_year_goodreads",
    "image_url": "cover_url_goodreads"
})

# Normalize titles and authors in both datasets
df_main["title_clean"] = df_main["title"].str.lower().str.strip()
df_main["author_clean"] = df_main["author"].str.lower().str.strip()

df_goodreads["title_clean"] = df_goodreads["title"].str.lower().str.strip()
df_goodreads["author_clean"] = df_goodreads["author"].str.lower().str.strip()

print("Columns prepared for merging:")
print(df_goodreads.head(3))


## Step 4 — Merge Datasets (Left Join by Title & Author)

In this step, we merge our main dataset (`books_clustered_final.csv`) with the Kaggle Goodreads dataset (`books.csv`)
using the normalized columns `title_clean` and `author_clean` as join keys.

This allows us to enrich our dataset with:
- More accurate average ratings from Goodreads
- Total number of ratings (`ratings_count_goodreads`)
- Publication year
- Cover image URL

We use a **left join** to keep all entries from our main dataset.


In [ ]:
# ============================================================
# Step 4 — Merge Datasets (Left Join by Title & Author)
# ============================================================

# Perform left join
df_merged = pd.merge(
    df_main,
    df_goodreads[
        [
            "title_clean",
            "author_clean",
            "avg_rating_goodreads",
            "ratings_count_goodreads",
            "published_year_goodreads",
            "cover_url_goodreads"
        ]
    ],
    on=["title_clean", "author_clean"],
    how="left"
)

print(f"Merge completed: {df_merged.shape}")

# Display sample of enriched data
df_merged[
    ["title", "author", "avg_rating", "avg_rating_goodreads", "ratings_count_goodreads"]
].head(10)


## Step 5 — Replace Imputed Ratings and Save Enriched Dataset

In this step, we replace the imputed values from our main dataset
with the real Goodreads data obtained from the merge.

Specifically:
- Replace `avg_rating` values equal to 4.11 with the Goodreads rating when available.
- Add the Goodreads `ratings_count` as a new feature.
- Save the enriched dataset as `books_final_enriched.csv` in the `data/clean` folder.


In [ ]:
# ============================================================
# Step 5 — Safely Replace Imputed Ratings and Save Enriched Dataset
# ============================================================

from functions import save_dataset
from pathlib import Path

# --- Create a copy to be safe ---
df_enriched = df_merged.copy()

# Replace only imputed ratings (4.11) with Goodreads ratings when available
mask_replace = (
    df_enriched["avg_rating"].round(2) == 4.11
) & (df_enriched["avg_rating_goodreads"].notna())

df_enriched.loc[mask_replace, "avg_rating"] = df_enriched.loc[
    mask_replace, "avg_rating_goodreads"
]

# Keep Goodreads ratings_count as a new column (optional feature)
df_enriched["ratings_count"] = df_enriched["ratings_count_goodreads"]

# Remove helper columns but keep your core structure intact
df_enriched = df_enriched.drop(columns=["avg_rating_goodreads", "ratings_count_goodreads"])

# Save the enriched dataset
output_path = Path("..") / "data" / "clean" / "books_final_enriched.csv"
save_dataset(df_enriched, output_path)

# --- Summary ---
print("✅ Enriched dataset saved safely → books_final_enriched.csv")
print(f"Ratings replaced (4.11 → Goodreads): {mask_replace.sum()}")
print(df_enriched[["title", "author", "avg_rating", "ratings_count"]].head(10))


In [ ]:
## Step 6 — Summary & Quality Check

In this final step, we evaluate how much the dataset improved after enrichment.

We will:
- Count how many books had their imputed `avg_rating` (4.11) replaced with real Goodreads values.
- Compare the average rating before and after enrichment.
- Show basic statistics for the new `ratings_count` feature.
